# Initial AI Agent PPO w/o
NN (working) MR2 Drag Disc

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 384
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 3,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 42, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 42,
  'trackChoice' : 1, # 0 is HS, 1 is 400m
  'carChoice' : 0, # 0 is MR2, 1 is Supra, 2 is Civic
  'rewardMode' : "complex"
}

In [3]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [4]:
import ray
ray.shutdown()
ray.init()

2023-10-21 10:51:01,541	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        #batch_mode="completed_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=384,
        model={
            "custom_model": "GTCustomNet.GTCustomNet",
            "custom_model_config": {
                "fcnet_hiddens":[256,256],
                }
        },
        _enable_learner_api=False
        )
    .rl_module( _enable_rl_module_api=False)
)

In [6]:
# import json

# class CustomEncoder(json.JSONEncoder):
#     def default(self, obj):
#         try:
#             json.dumps(obj)
#         except TypeError:
#             return "Not serializable object!"

#         return obj

# config_dict = config.to_dict()

# print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

In [7]:
algo = config.build()

2023-10-21 10:51:07,286	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=1736) GT Real Time instantiated
(RolloutWorker pid=1736) GT AI Server instantiated for rtgym
(RolloutWorker pid=1736) starting up on localhost port 9999
(RolloutWorker pid=1736) Waiting for a connection


(RolloutWorker pid=1736) 2023-10-21 10:51:16,310	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!
2023-10-21 10:51:16,461	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=1736) Connection from ('127.0.0.1', 52257)
(RolloutWorker pid=1736) J:\git\TensorFlowPSX\Py\dragSpaced.csv
(RolloutWorker pid=1736) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2
(RolloutWorker pid=1736) I found a BOX
(RolloutWorker pid=1736) Box(0.0, 3.0, (1,), float64)
(RolloutWorker pid=1736) I found a BOX
(RolloutWorker pid=1736) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=1736) I found a BOX
(RolloutWorker pid=1736) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=1736) I found a BOX
(RolloutWorker pid=1736) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=1736) I found Cov2d layers
(RolloutWorker pid=1736) Box(0, 255, (42, 42, 1), uint8)
(RolloutWorker pid=1736) I found Cov2d layers
(RolloutWorker pid=1736) Box(0, 255, (42, 42, 1), uint8)
I found a BOX
Box(0.0, 3.0, (1,), float64)
(RolloutWorker pid=1736) I found Cov2d layers
(RolloutWorker pid=1736) Box(0, 255, (42, 42, 1), uint8)
(RolloutWorker pid=1736) I found a BOX
(RolloutWorker pid=

2023-10-21 10:51:19,151	INFO trainable.py:172 -- Trainable.setup took 11.865 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
algo.get_policy().model

GTCustomNet(
  (cnns): ModuleDict(
    (4): VisionNetwork(
      (_convs): Sequential(
        (0): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((1, 1, 1, 1))
            (1): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2))
          )
        )
        (1): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((1, 2, 1, 2))
            (1): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2))
          )
        )
        (2): SlimConv2d(
          (_model): Sequential(
            (0): Conv2d(32, 256, kernel_size=(11, 11), stride=(1, 1))
          )
        )
        (3): Flatten(start_dim=1, end_dim=-1)
      )
      (_value_branch_separate): Sequential(
        (0): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((1, 1, 1, 1))
            (1): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2))
          )
        )
        (1): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((1, 2, 1, 2))
         

In [9]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=1736) reset triggered
(RolloutWorker pid=1736) reload save for track : 72
(RolloutWorker pid=1736) reset triggered
(RolloutWorker pid=1736) reload save for track : 72


(RolloutWorker pid=1736) C:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1767124999997307
(RolloutWorker pid=1736)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0


(RolloutWorker pid=1736) C:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 3.389690999999999
(RolloutWorker pid=1736)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=1736) reset triggered
(RolloutWorker pid=1736) reload save for track : 72


...

(RolloutWorker pid=1736) reset triggered
(RolloutWorker pid=1736) reload save for track : 72
Loop:  442


(RolloutWorker pid=1736) C:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 2.9948930002083216
(RolloutWorker pid=1736)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=1736) reset triggered
(RolloutWorker pid=1736) reload save for track : 72


(RolloutWorker pid=1736) C:\Users\nadir\anaconda3\envs\GTRay2.4.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.13501130007171014
(RolloutWorker pid=1736)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


KeyboardInterrupt: 

#Works!

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-15_16-55-415yyb54u3/checkpoint_001221")

In [ ]:
result = algo.train() #single try

In [ ]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()
